# Graphs and sessions in TensorFlow

Source: https://www.tensorflow.org/guide/graphs

In Teonsorflow, a graph is the representation of operations, while a session is an executor for a graph. Before being executed by a session, a graph is only a blueprint of the operations, which in turn return no value until they are executed.

In [1]:
import tensorflow as tf
from pprint import pprint

Defining a constant operation (a constant). This get added to the Tensorflow default graph unless otherwise specified.

Resetting the default graph ensures that if we run cells multiple times, operations are not added more than once.

In [25]:
tf.reset_default_graph()

c_0 = tf.constant(43., name='c')

print("Operations currently in the default graph:")
print(tf.get_default_graph().get_operations())

print("\nValue of operation c_0 without execution:")
print(c_0)

Operations currently in the default graph:
[<tf.Operation 'c' type=Const>]

Value of operation c_0 without execution:
Tensor("c:0", shape=(), dtype=float32)


Operations can be defined within a particular name scope to add clarity to their names. Again, these are added to the default graph.

In [26]:
tf.reset_default_graph()

with tf.name_scope("test_scope"):
    t1 = tf.constant(1., name="t1")
    t2 = tf.constant(2., name="t2")

print("Operations currently in the default graph:")
pprint(tf.get_default_graph().get_operations())

Operations currently in the default graph:
[<tf.Operation 'test_scope/t1' type=Const>,
 <tf.Operation 'test_scope/t2' type=Const>]


To get the values of operations, a session is needed. A session is a connection between Python and the underlying C++ runtime Tensorflow uses to execute operations in graphs. A session "owns physical resources": when one is defined, it can be indicated which device should execute the code (CPU, GPU, ...) in its options.

If not otherwise specified in its options, a session executes operations in the default graph.

In [31]:
tf.reset_default_graph()

x = tf.constant([[1., 0.], [0., -1.]])
y = tf.constant([[1., 1.], [1., 1.]])
xtimesy = tf.matmul(x, y)
z = tf.nn.softmax(xtimesy)

sess = tf.Session()

print("Tensor returned by operation xtimesy:")
print(sess.run(xtimesy))

print("\nTensor returned by operation z:")
print(sess.run(z))

Tensor returned by operation xtimesy:
[[ 1.  1.]
 [-1. -1.]]

Tensor returned by operation z:
[[0.5 0.5]
 [0.5 0.5]]


Sessions can also be defined in contexts.

In [37]:
tf.reset_default_graph()

m = tf.constant([1, 1, 1], shape=[1, 3])

with tf.Session() as sess_c:
    print(sess_c.run(m))

[[1 1 1]]


Tensors can be passed values dynamically using placeholders.

In [38]:
tf.reset_default_graph()

dummy = tf.placeholder(dtype=tf.float32, shape=[3, 3])

v = tf.constant([2., 1., 14.], shape=[3, 1])

dv = tf.matmul(dummy, v)

with tf.Session() as sess:
    print(sess.run(
        dv,
        {dummy: [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]}
    ))

[[ 2.]
 [ 1.]
 [14.]]


We can explicitly define multiple graphs and assign operations to them.

In [42]:
g_1 = tf.Graph()

with g_1.as_default():
    t1 = tf.constant([1, 2, 3], shape=[3, 1], name="t1_in_g1")
    
    print("While using g_1 as default graph, list the operations:")
    print(tf.get_default_graph().get_operations())
    
g_2 = tf.Graph()

with g_2.as_default():
    t2 = tf.constant([1, 1], shape=[1, 2], name="t2_in_g2")
    
    print("\nWhile using g_2 as default graph, list the operations:")
    print(tf.get_default_graph().get_operations())

print(
    "\nWithout any context defining a particular graph as default, "
    "list operations in the original default graph:"
)
pprint(tf.get_default_graph().get_operations())

While using g_1 as default graph, list the operations:
[<tf.Operation 't1_in_g1' type=Const>]

While using g_2 as default graph, list the operations:
[<tf.Operation 't2_in_g2' type=Const>]

Without any context defining a particular graph as default, list operations in the original default graph:
[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'MatMul' type=MatMul>]


Different sessions running different graphs can be defined explicitly.

In [43]:
sess_1 = tf.Session(graph=g_1)

print("Session sess_1 can run operations within graph g_1:")
print(t1.name, ":")
print(sess_1.run(t1))

print("\n...but not operations within another graph: ", end="")

try:
    sess_1.run(t2)
except ValueError:
    
    print("tensor t2 is not an operation within graph g_1")
    
sess_2 = tf.Session(graph=g_2)

print("\nSession sess_2 can run operations within graph g_2:")
print(t2.name, ":")
print(sess_2.run(t2))

Session sess_1 can run operations within graph g_1:
t1_in_g1:0 :
[[1]
 [2]
 [3]]

...but not operations within another graph: tensor t2 is not an operation within graph g_1

Session sess_2 can run operations within graph g_2:
t2_in_g2:0 :
[[1 1]]


In [161]:
tf.reset_default_graph()

t_rand = tf.random.uniform(shape=[3, 3], dtype=tf.float32, name="random_3by3")
t_rand_trace = tf.linalg.trace(t_rand)
t_rand_det = tf.linalg.det(t_rand)

with tf.Session() as sess:
    t_rand_eval, trace_eval, det_eval = sess.run((
        t_rand,
        t_rand_trace,
        t_rand_det
    ))
    
print("Random 3x3 tensor:")
print(t_rand_eval)

print("\nIts trace:")
print(trace_eval)

print("\nIts determinant:")
print(det_eval)

Random 3x3 tensor:
[[0.46885467 0.19841158 0.4124316 ]
 [0.39023232 0.27491236 0.06212771]
 [0.6543205  0.6714376  0.95432174]]

Its trace:
1.6980888

Its determinant:
0.071499325
